Mestrado em Modelagem Matematica da Informacao
----------------------------------------------
Disciplina: Modelagem e Mineracao de Dados
------------------------------------------

Master Program - Mathematical Modeling of Information
-----------------------------------------------------
Course: Data Mining and Modeling
--------------------------------

Professor: Renato Rocha Souza

### Topic: Advanced Scraping

Information on the Python Packages used: 

https://pypi.python.org/pypi/mechanize/  
http://wwwsearch.sourceforge.net/mechanize/  
http://www.pythonforbeginners.com/mechanize/browsing-in-python-with-mechanize  
http://www.pythonforbeginners.com/cheatsheet/python-mechanize-cheat-sheet  
http://www.seleniumhq.org/  
http://stackoverflow.com/questions/8255929/running-webdriver-chrome-with-selenium  
https://pypi.python.org/pypi/PyVirtualDisplay  
http://coreygoldberg.blogspot.com.br/2011/06/python-headless-selenium-webdriver.html  

In [27]:
import re
import json
import os
import urllib 
import urllib2
import requests
import mechanize
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
#from pyvirtualdisplay import Display

In [2]:
# Browser
br = mechanize.Browser()

# User-Agent (this is cheating, ok?)
br.addheaders = [('User-agent', 
                  'Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.1) Gecko/2008071615 Fedora/3.0.1-1.fc9 Firefox/3.0.1')]

In [3]:
url='http://www.amazon.com/Hundred-Solitude-Gabriel-Garcia-Marquez/dp/006112009X/'

# fetch the page
response = br.open(url)
print br.title()

One Hundred Years of Solitude: Gabriel Garcia Marquez: 9780061120091: Amazon.com: Books


In [4]:
print response.geturl() # the URL the browser is currently at

http://www.amazon.com/Hundred-Solitude-Gabriel-Garcia-Marquez/dp/006112009X/


In [5]:
print response.info() # HTML headers 

Date: Tue, 09 Dec 2014 18:53:43 GMT
Server: Server
pragma: no-cache
x-amz-id-1: 04V58AT585DPCEKQV8SK
p3p: policyref="http://www.amazon.com/w3c/p3p.xml",CP="CAO DSP LAW CUR ADM IVAo IVDo CONo OTPo OUR DELi PUBi OTRi BUS PHY ONL UNI PUR FIN COM NAV INT DEM CNT STA HEA PRE LOC GOV OTC "
x-sap-pg: book_display_on_website
cache-control: no-cache
x-frame-options: SAMEORIGIN
expires: -1
x-amz-id-2: crP8ssRSi3e1wYiTtivcIIdMuverjVGFnx/Z9CRVSsEjuXLvfnbGjsVe594nm8ZN
x-ua-compatible: IE=edge
Vary: Accept-Encoding,User-Agent
Content-Type: text/html; charset=UTF-8
Set-cookie: x-wl-uid=1SZijsUGCAaiDlWZl/q15ZfLl/mJR1uiyZjA78HXu5nHP6IzEkh6BwqGhWVk3l2q63rpcTT5eVzU=; path=/; domain=.amazon.com; expires=Tue, 01-Jan-2036 08:00:01 GMT
Set-cookie: session-id-time=2082787201l; path=/; domain=.amazon.com; expires=Tue, 01-Jan-2036 08:00:01 GMT
Set-cookie: session-id=182-1492422-7064656; path=/; domain=.amazon.com; expires=Tue, 01-Jan-2036 08:00:01 GMT
Transfer-Encoding: chunked



In [6]:
response = br.follow_link(text_regex=r"Gabriel Garcia Marquez")
print "Review Page:"
print br.title()
print response.geturl() # the URL the browser is currently at

Review Page:
Amazon.com: Gabriel Garcia Marquez: Books, Biography, Blog, Audiobooks, Kindle
http://www.amazon.com/Gabriel-Garcia-Marquez/e/B000AQ1JWC/ref=dp_byline_cont_pop_book_1


If you need to fill out and submit Web Forms, mechanize [is particularly useful](http://wwwsearch.sourceforge.net/mechanize/forms.html):

In [7]:
br.select_form(name="site-search") # select the form
form = br.form # put it in a variable to make it easier to work with

form['url'] = ['search-alias=stripbooks']
form['field-keywords'] = 'Gabriel Garcia Marquez'
br.submit()

print br.title()
print br.geturl()

Amazon.com: Gabriel Garcia Marquez: Books
http://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks&field-keywords=Gabriel+Garcia+Marquez


Some options that can come in handy:

    # Browser options
    br.set_proxies({"http": "joe:password@myproxy.example.com:3128"}) # if you need to go through a proxy
    br.set_handle_redirect(True)
    br.set_handle_referer(True)
    br.set_handle_robots(False) # some pages do not want to be scraped (http://www.robotstxt.org/robotstxt.html)

    # Follows refresh 0 but not hangs on refresh > 0
    br.set_handle_refresh(mechanize._http.HTTPRefreshProcessor(), max_time=1)

    # Want debugging messages?
    br.set_debug_http(False)
    br.set_debug_redirects(False)
    br.set_debug_responses(False)

Which brings us to another way of submitting (some) forms: directly through a URL. 

Lets [break down](http://www.mattcutts.com/blog/seo-glossary-url-definitions/): [http://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks&field-keywords=Gabriel+Garcia+Marquez](http://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks&field-keywords=Gabriel+Garcia+Marquez)

You use them everyday, now it is time to understand the parts: 

* `http://` is the "protocol". [http](http://en.wikipedia.org/wiki/Http) is the standard one, you will also see [https](http://en.wikipedia.org/wiki/Https) which is the "secure" one
* `www.amazon.com` is the [domain name](http://en.wikipedia.org/wiki/Domain_name). It can have more parts, but it is everything before the first `/`
* `s/ref=nb_sb_noss` is the path. It is not typical to have =, but acceptable. This specifies the page within the domain name. It can/be/many/parts/. It is everything between the first / and the first ?
* `url=search-alias%3Dstripbooks&field-keywords=Gabriel+Garcia+Marquez` are the query parameters. Basically the results of form submissions. 

Lets look at the parameters in more detail: 

* parameters are separated by `&`, so we have: 
 * url=search-alias%3Dstripbooks
 * field-keywords=Gabriel+Garcia+Marquez
* parameter names are preceded by an `=`, so we have: 
 * url
 * field-keywords
* parameter values are "url escaped" (sometimes known as [percent encoding](http://en.wikipedia.org/wiki/Url_escape))
 * search-alias%3Dstripbooks => search-alias=stripbooks (this allows you to have = and other special symbols)
 * Gabriel+Garcia+Marquez (sometimes Gabriel%20Garcia%20Marquez) => Gabriel Garcia Marquez

Fortunately, you can get python to figure it all out for you: 

In [8]:
# make a dictionary of your parameters
params = {'url': 'search-alias=stripbooks', 'field-keywords': 'Vargas Losa'}
print urllib.urlencode(params)

url=search-alias%3Dstripbooks&field-keywords=Vargas+Losa


In [9]:
# put it together with the rest of the URL
url = 'http://www.amazon.com/s/ref=nb_sb_noss' + '?' + urllib.urlencode(params)
print url

http://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks&field-keywords=Vargas+Losa


**Exercise 1:**

Write a loop that follows the "next" link 10 times. 

_Hint:_ Spend some time reading through the examples on the [Mechanize Homepage](http://wwwsearch.sourceforge.net/mechanize/). The documentation is not as thorough as for BeautifulSoup, but there are enough examples on follow links to do this assignment. 

In [10]:
#for i in range(0,10):
#    ## your code here
#    br.geturl()
  
# answer: 
#br.follow_link(url_regex=r'.*btm_link_next_.*')

### Working with APIs ###

Working with (most) API's is not very different. <b>A</b>pplication <b>P</b>rogramming <b>I</b>nterfaces are basically just websites that return structured data instead of an HTML page. 

Lets have a look at the [World Bank API](http://data.worldbank.org/node/11). Like most of the APIs you'll ever use, it has a [RESTful API](http://stackoverflow.com/questions/671118/what-exactly-is-restful-programming) and parameter (argument) based endpoints.

Again, the only difference with using an API from scraping is that the data comes back in a machine-readable format, not just HTML. The two most common formats are XML and JSON. JSON is actually very similar to the syntax for python dictionaries and there is a [json module](http://docs.python.org/2/library/json.html) to help us convert JSON into python.

Lets look at the JSON module:

In [11]:
#api_response = '[{"page":1,"pages":1,"per_page":"50","total":1},[{"id":"BRA","iso2Code":"BR","name":"Brazil","region":{"id":"LCN","value":"Latin America & Caribbean (all income levels)"},"adminregion":{"id":"LAC","value":"Latin America & Caribbean (developing only)"},"incomeLevel":{"id":"UMC","value":"Upper middle income"},"lendingType":{"id":"IBD","value":"IBRD"},"capitalCity":"Brasilia","longitude":"-47.9292","latitude":"-15.7801"}]]'
api_response = urllib2.urlopen('http://api.worldbank.org/countries/BRA?format=json').read()

# loads (load string) will convert it to a Python object we can use
json_response = json.loads(api_response)

# we have a list, with a list some lists or dicts inside
print json_response

[{u'per_page': u'50', u'total': 1, u'page': 1, u'pages': 1}, [{u'longitude': u'-47.9292', u'name': u'Brazil', u'region': {u'id': u'LCN', u'value': u'Latin America & Caribbean (all income levels)'}, u'adminregion': {u'id': u'LAC', u'value': u'Latin America & Caribbean (developing only)'}, u'iso2Code': u'BR', u'capitalCity': u'Brasilia', u'latitude': u'-15.7801', u'incomeLevel': {u'id': u'UMC', u'value': u'Upper middle income'}, u'id': u'BRA', u'lendingType': {u'id': u'IBD', u'value': u'IBRD'}}]]


In [12]:
# we can grab the data we want from our list
brazil_data = json_response[1][0]
print brazil_data['capitalCity']

Brasilia


Example: Use the World Bank API to find the [GNI per Capita](http://data.worldbank.org/indicator/NY.GNP.PCAP.CD/) of Argentina, Brazil, Mexico and Canada

In [13]:
for country in ['usa', 'bra', 'mex', 'can']:
    url = 'http://api.worldbank.org/countries/' + country + '/indicators/NY.GNP.PCAP.CD?date=2012&format=json'
    content = urllib2.urlopen(url).read()
    response = json.loads(content)
    print response[1][0]['value']

52350
11640
9720
50650


**Exercise 2:**

Get the School enrollment data for the same countries

### Time to take a drive

The last option for web scraping is to write some code to drive a browser. The easiest option to do that is to use [Selenium](http://www.seleniumhq.org/). Selenium was actually intended to help do automated testing of Websites, but it works well for our purposes. 

While everything we've seen so far could be put on a server to run and only requires python, Selenium will require your actual browser. It will use the browser itself to fetch and render the pages. 

This is a great option in two scenarios: 

1. If the page you want to parse requires a lot of javascript or something else that does not let urllib or mechanize to work with the page. Some sites do this on purpose to prevent scraping. 
2. If you have to do a lot of navigating around a page to get to the information you want to scrape

In [14]:
#Choosing your browser:

# iPhone
#driver = webdriver.Remote(browser_name="iphone", command_executor='http://172.24.101.36:3001/hub')
 
# Android
#driver = webdriver.Remote(browser_name="android", command_executor='http://127.0.0.1:8080/hub')
 
# Google Chrome 
#driver = webdriver.Chrome()
 
# Firefox 
driver = webdriver.Firefox()

In [15]:
driver.get("http://www.google.com")
print driver.title

Google


In [16]:
print driver.page_source.encode('utf-8')

<!DOCTYPE html>
<html xmlns="http://www.w3.org/1999/xhtml" lang="pt" itemtype="http://schema.org/WebPage" itemscope=""><head><meta itemprop="image" content="/images/google_favicon_128.png" /><title>Google</title>   <script src="https://apis.google.com/_/scs/abc-static/_/js/k=gapi.gapi.en.edEzvWMtWYM.O/m=gapi_iframes,googleapis_client,plusone/rt=j/sv=1/d=1/ed=1/rs=AItRSTNodbd3fm2h0vWoq_h0iFXYtPUsXw/cb=gapi.loaded_0" async=""></script><script>(function(){window.google={kEI:'U0WHVL_bDsiqgwSShIOwCw',kEXPI:'3700301,4011550,4011552,4011557,4011559,4013920,4014789,4016824,4017578,4020346,4020562,4021073,4021587,4021597,4023217,4023567,4023678,4023709,4025182,4025285,4025828,4025836,4026125,8500394,8500852,8500882,10200083,10200716,10200828,10200835,10200844,10200882',authuser:0,j:{en:1,bv:21,pm:'p',u:'c9c918f0',qbp:0,rre:false},kSID:'U0WHVL_bDsiqgwSShIOwCw'};google.kHL='pt';})();(function(){google.lc=[];google.li=0;google.getEI=function(a){for(var b;a&amp;&amp;(!a.getAttribute||!(b=a.getAttri

In [17]:
driver.get('http://codepad.org')

In [18]:
# Select the Python language option
python_link = driver.find_elements_by_xpath("//input[@name='lang' and @value='Python']")[0]
python_link.click()

In [19]:
# Enter some text!
text_area = driver.find_element_by_id('textarea')
text_area.send_keys("print 'Hello,' + ' World!'")

In [20]:
# Submit the form!
submit_button = driver.find_element_by_name('submit')
submit_button.click()

In [21]:
driver.quit()

In [22]:
driver = webdriver.Firefox()
driver.get("http://www.python.org")

In [23]:
assert "Python" in driver.title

In [24]:
#http://selenium-python.readthedocs.org/en/latest/locating-elements.html#locating-elements
elem = driver.find_element_by_name("q")
elem.clear()
elem.send_keys("pycon")
elem.send_keys(Keys.RETURN)

In [25]:
assert "No results found." not in driver.page_source

In [26]:
driver.close()

**Exercise 3:**

Open a browser and "drive" it from the EMAp home page until this discipline's page: 
http://emap.fgv.br/pos-graduacao/disciplina/modelagem-mineracao-dados.html  